In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as Funct
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
import csv

sparkSession = SparkSession.builder.appName("myApp").getOrCreate()

In [2]:
#Exercise 1.5 (Loading the dataset into DataFrames)
#users_libraries.txt
df = sparkSession.read.csv(path="./mod_users_libraries.txt", sep=';')
df.show(2, truncate=True)

+--------------------+--------------------+
|                 _c0|                 _c1|
+--------------------+--------------------+
|28d3f81251d94b097...|3929762,503574,58...|
|d0c9aaa788153daea...|2080631,6343346,5...|
+--------------------+--------------------+
only showing top 2 rows



In [3]:
#Exercise 1.5 (Loading the dataset into DataFrames)
#users_libraries.txt
userLibRDD = sparkSession.sparkContext.textFile("./mod_users_libraries.txt")\
                .map(lambda x: (x.split(';')[0], x.split(';')[1]))\
                .map(lambda x: (x[0], x[1].split(',')))\
                .flatMapValues(lambda x: x)
#userLibRDD.take(100)
schemaString = "user_hash_id paper_id"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
userLibDF = sparkSession.createDataFrame(userLibRDD, schema)

userLibDF.show(25, truncate=True)


+--------------------+--------+
|        user_hash_id|paper_id|
+--------------------+--------+
|28d3f81251d94b097...| 3929762|
|28d3f81251d94b097...|  503574|
|28d3f81251d94b097...| 5819422|
|28d3f81251d94b097...| 4238883|
|28d3f81251d94b097...| 5788061|
|28d3f81251d94b097...|  462949|
|28d3f81251d94b097...|  635215|
|28d3f81251d94b097...|  635216|
|28d3f81251d94b097...| 4810441|
|28d3f81251d94b097...| 3481823|
|28d3f81251d94b097...| 4165233|
|28d3f81251d94b097...| 3366480|
|28d3f81251d94b097...| 5984302|
|28d3f81251d94b097...| 4238942|
|28d3f81251d94b097...| 5490453|
|28d3f81251d94b097...| 4636156|
|28d3f81251d94b097...| 5996865|
|28d3f81251d94b097...| 4194836|
|28d3f81251d94b097...| 5828780|
|28d3f81251d94b097...| 4450195|
|d0c9aaa788153daea...| 2080631|
|d0c9aaa788153daea...| 6343346|
|d0c9aaa788153daea...| 5184704|
|d0c9aaa788153daea...| 7756088|
|d0c9aaa788153daea...| 2653863|
+--------------------+--------+
only showing top 25 rows



In [14]:
#Exercise 1.5 (Loading the dataset into DataFrames)
#papers.csv
columns = ['paper_id', 'type', 'journal', 'book_title', 'series', 'publisher', 'pages', 'volume', 'number', 'year', 'month', 'postedate', 'address', 'title', 'abstract']
papersDF = sparkSession.read.load("./papers.csv", format="csv", sep=",", inferSchema="true", quote='"', header="false").toDF(*columns)
papersDF.show(2, truncate=True)


+--------+-------+--------------------+----------+------+---------+-----+------+------+----+-----+-------------------+-------+--------------------+--------------------+
|paper_id|   type|             journal|book_title|series|publisher|pages|volume|number|year|month|          postedate|address|               title|            abstract|
+--------+-------+--------------------+----------+------+---------+-----+------+------+----+-----+-------------------+-------+--------------------+--------------------+
|   80546|article|biology and philo...|      null|  null|     null|   17|    19|     2|2004|  mar|2005-01-26 21:35:21|   null|the arbitrariness...|the genetic code ...|
| 5842862|article|      molecular cell|      null|  null| elsevier|    2|    35|     6|2009|  sep|2009-09-30 17:11:23|   null|how to choose a g...|choosing good pro...|
+--------+-------+--------------------+----------+------+---------+-----+------+------+----+-----+-------------------+-------+--------------------+--------

In [15]:
#Exercise 1.5 (Loading the dataset into DataFrames)
#stopwords_en.txt
stopWordsDF = sparkSession.read.load("./stopwords_en.txt", format="text", sep=" ", inferSchema="true", header="false").toDF('stop_word')
stopWordsDF.show(5, truncate=True)


+---------+
|stop_word|
+---------+
|        a|
|     able|
|    about|
|    above|
|according|
+---------+
only showing top 5 rows



In [6]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 a) Using DataFrames (Basic Analysis for Recommender Systems)
#Number of distinct users
userLibDF.select("user_hash_id").distinct().count()


1000

In [7]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 a) Using DataFrames (Basic Analysis for Recommender Systems)
#Number of distinct items
papersDF.select("paper_id").distinct().count()

172079

In [8]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 a) Using DataFrames (Basic Analysis for Recommender Systems)
#Number of ratings
userLibDF.select("paper_id").count()

30149

In [9]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 b) Using DataFrames (Basic Analysis for Recommender Systems)
#Min number of ratings a user has given
userLibDF.select("user_hash_id").groupBy("user_hash_id").count().agg(Funct.min("count")).show()

+----------+
|min(count)|
+----------+
|         1|
+----------+



In [10]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 c) Using DataFrames (Basic Analysis for Recommender Systems)
#Max number of ratings a user has given
userLibDF.select("user_hash_id").groupBy("user_hash_id").count().agg(Funct.max("count")).show()

+----------+
|max(count)|
+----------+
|       950|
+----------+



In [11]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 d) Using DataFrames (Basic Analysis for Recommender Systems)
#Average number of ratings of users
userLibDF.select("user_hash_id").groupBy("user_hash_id").count().agg(Funct.mean("count")).show()

+----------+
|avg(count)|
+----------+
|    30.149|
+----------+



In [12]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 e) Using DataFrames (Basic Analysis for Recommender Systems)
#Standard deviation for ratings of users
userLibDF.select("user_hash_id").groupBy("user_hash_id").count().agg(Funct.stddev("count")).show()

+------------------+
|stddev_samp(count)|
+------------------+
|  71.4915368336619|
+------------------+



In [13]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 f) Using DataFrames (Basic Analysis for Recommender Systems)
#Min number of ratings an item has received
userLibDF.select("paper_id").groupBy("paper_id").count().agg(Funct.min("count")).show()

+----------+
|min(count)|
+----------+
|         1|
+----------+



In [14]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 g) Using DataFrames (Basic Analysis for Recommender Systems)
#Max number of ratings an item has received
userLibDF.select("paper_id").groupBy("paper_id").count().agg(Funct.max("count")).show()

+----------+
|max(count)|
+----------+
|        30|
+----------+



In [15]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 h) Using DataFrames (Basic Analysis for Recommender Systems)
#Average number of ratings of items
userLibDF.select("paper_id").groupBy("paper_id").count().agg(Funct.mean("count")).show()

+------------------+
|        avg(count)|
+------------------+
|1.1188673643583462|
+------------------+



In [16]:
#Exercise 1.6 (Tasks on top of DataFrames)
#Exercise 1.4 i) Using DataFrames (Basic Analysis for Recommender Systems)
#Average number of ratings of items
userLibDF.select("paper_id").groupBy("paper_id").count().agg(Funct.stddev("count")).show()

+-------------------+
| stddev_samp(count)|
+-------------------+
|0.47627972494626164|
+-------------------+



In [20]:
#Exercise 1.6
#Exercise 1.3 Using Dataframes (Joining collections)
#Joining the userLib dataframe with the papersDf dataframe over paper_id
#Dropping any row that has null value in the abstract column
joinedDF = userLibDF.join(papersDF, userLibDF.paper_id == papersDF.paper_id, "inner")\
                        .select(userLibDF.user_hash_id,\
                                Funct.explode(Funct.split(papersDF.abstract," "))\
                                .alias("word"))\
                        .na.drop("any")

joinedDF.show(10, truncate=True)
#joinedDF.count()

+--------------------+-------+
|        user_hash_id|   word|
+--------------------+-------+
|a6f5b862e26386cec...|   this|
|a6f5b862e26386cec...|  paper|
|a6f5b862e26386cec...|     is|
|a6f5b862e26386cec...|    the|
|a6f5b862e26386cec...|  fifth|
|a6f5b862e26386cec...|     in|
|a6f5b862e26386cec...|      a|
|a6f5b862e26386cec...| series|
|a6f5b862e26386cec...|     of|
|a6f5b862e26386cec...|studies|
+--------------------+-------+
only showing top 10 rows



In [22]:
#Exercise 1.6
#Exercise 1.3 Using Dataframes (Joining collections)
#Subtracting the stop words from the user and abstract words dataframe
withoutStpWrdDF = joinedDF.join(stopWordsDF, joinedDF.word==stopWordsDF.stop_word, how="left_anti")
#withoutStpWrdDF.count()
withoutStpWrdDF.show(20, truncate=True)

+--------------------+----------------+
|        user_hash_id|            word|
+--------------------+----------------+
|a6f5b862e26386cec...|           paper|
|a6f5b862e26386cec...|          series|
|a6f5b862e26386cec...|         studies|
|a6f5b862e26386cec...|       emanating|
|a6f5b862e26386cec...|            {uk}|
|a6f5b862e26386cec...|           joint|
|a6f5b862e26386cec...|     information|
|a6f5b862e26386cec...|         systems|
|a6f5b862e26386cec...|       committee|
|a6f5b862e26386cec...| ({jisc})-funded|
|a6f5b862e26386cec...|         {romeo}|
|a6f5b862e26386cec...|         project|
|a6f5b862e26386cec...|         (rights|
|a6f5b862e26386cec...|        metadata|
|a6f5b862e26386cec...|open-archiving).|
|a6f5b862e26386cec...|           paper|
|a6f5b862e26386cec...|         reports|
|a6f5b862e26386cec...|         results|
|a6f5b862e26386cec...|         surveys|
|a6f5b862e26386cec...|           {oai}|
+--------------------+----------------+
only showing top 20 rows



In [46]:
#Exercise 1.6
#Exercise 1.3 Using Dataframes (Joining collections)
#Grouping the joinedDf dataframe over 'user_hash_id' and 'word' columns
#Sorting in ascending order the result w.r.t the 'user_hash_id' column
groupedDF = joinedDF.groupBy("user_hash_id", "word")\
                    .agg(Funct.count("word").alias("word_count"))\
                    .sort(Funct.asc("user_hash_id"))

groupedDF.show(10, truncate=True)

+--------------------+---------+----------+
|        user_hash_id|     word|word_count|
+--------------------+---------+----------+
|003840e52f6a43bc5...| striking|         1|
|003840e52f6a43bc5...|politics.|         1|
|003840e52f6a43bc5...|   brings|         1|
|003840e52f6a43bc5...| policies|         1|
|003840e52f6a43bc5...|    these|         2|
|003840e52f6a43bc5...|    life.|         1|
|003840e52f6a43bc5...|  meaning|         1|
|003840e52f6a43bc5...|disasters|         1|
|003840e52f6a43bc5...|   framed|         1|
|003840e52f6a43bc5...| american|         2|
+--------------------+---------+----------+
only showing top 10 rows



In [56]:
#Exercise 1.6
#Exercise 1.3 Using Dataframes (Joining collections)
#Creating a window and using 'rank' function over the window to give a rank to the words per user and according 
#to the word count
window = Window.partitionBy("user_hash_id").orderBy(Funct.desc("word_count"))
rankedResultDF = groupedDF.withColumn("ranking", Funct.rank().over(window))
filteredRankedResDF = rankedResultDF.filter(rankedResultDF.ranking < 10).select("user_hash_id", "word", "ranking")
filteredRankedResDF.show(30, truncate=True)

+--------------------+---------+-------+
|        user_hash_id|     word|ranking|
+--------------------+---------+-------+
|003840e52f6a43bc5...|      the|      1|
|003840e52f6a43bc5...|       of|      2|
|003840e52f6a43bc5...|      and|      3|
|003840e52f6a43bc5...|       in|      4|
|003840e52f6a43bc5...|       to|      5|
|003840e52f6a43bc5...|        a|      6|
|003840e52f6a43bc5...|   social|      7|
|003840e52f6a43bc5...|     that|      8|
|003840e52f6a43bc5...|      war|      9|
|003840e52f6a43bc5...|       by|      9|
|003840e52f6a43bc5...|     this|      9|
|00be9ee3318c2cf69...|       of|      1|
|00be9ee3318c2cf69...|      the|      2|
|00be9ee3318c2cf69...|      and|      3|
|00be9ee3318c2cf69...|       in|      4|
|00be9ee3318c2cf69...|  {rhios}|      5|
|00be9ee3318c2cf69...|     that|      6|
|00be9ee3318c2cf69...|       to|      6|
|00be9ee3318c2cf69...|hospitals|      6|
|00be9ee3318c2cf69...|    {hie}|      6|
|00be9ee3318c2cf69...|       is|      6|
|00be9ee3318c2cf